[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

# N-BEATS TensorFlow Tutorial

**Paper:** Oreshkin, B. N., Carpov, D., Chapados, N., & Bengio, Y. (2019). N-BEATS: Neural basis expansion analysis for interpretable time series forecasting. [arXiv:1905.10437](https://arxiv.org/abs/1905.10437). 

**Data:** SILSO, World Data Center - Sunspot Number and Long-Term Solar Observations, Royal Observatory of Belgium, 1818 - 2021. https://wwwbis.sidc.be/silso/datafiles.

**Code:** TensorFlow (Python) implementation of N-BEATS model for univariate time series forecasting. https://github.com/flaviagiammarino/nbeats-tensorflow.

## 1. Set Up

Install the dependencies.

In [ ]:
!pip install pandas numpy tensorflow plotly optuna

Clone the repository.

In [ ]:
!pip install github-clone

In [ ]:
!ghclone https://github.com/flaviagiammarino/nbeats-tensorflow/tree/main/nbeats_tensorflow

Import the libraries.

In [ ]:
import optuna
import logging
import pandas as pd
import numpy as np
import tensorflow as tf
import plotly.graph_objects as go

In [ ]:
from nbeats_tensorflow.model import NBeats

Set the logging level.

In [ ]:
tf.get_logger().setLevel(logging.ERROR)
optuna.logging.set_verbosity(optuna.logging.INFO)

Fix the random seeds.

In [ ]:
TENSORFLOW_SEED = 0
OPTUNA_SEED = 0

## 2. Data

Download the data.

In [ ]:
df = pd.read_csv('https://wdc-silso-daily-sunspot-number.s3.eu-west-2.amazonaws.com/SN_d_tot_V2.0.csv', sep=';', header=None, usecols=[0, 1, 2, 4], names=['year', 'month', 'day', 'y'])

In [ ]:
df

Fill the missing values with zero.

In [ ]:
df.loc[df['y'] == -1, 'y'] = 0

Generate the timestamps.

In [ ]:
df['timestamp'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str) + '-' + df['day'].astype(str))

Drop the unnecessary columns.

In [ ]:
df = df[['timestamp', 'y']]

Downsample the data from daily to monthly.

In [ ]:
df = df.set_index('timestamp').resample('M')['y'].mean().reset_index()

In [ ]:
df

Plot the data.

In [ ]:
layout = dict(
    width=800,
    height=400,
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(t=40, b=20, l=20, r=20),
    xaxis=dict(
        type='date',
        nticks=20,
        tickfont=dict(
            color='#3a3a3a',
            size=10,
        ),
        linecolor='#d9d9d9',
        mirror=True,
        showgrid=False,
    ),
    yaxis=dict(
        tickfont=dict(
            color='#3a3a3a',
            size=10,
        ),
        linecolor='#d9d9d9',
        mirror=True,
        showgrid=False,
        zeroline=False,
    ),
)

data = go.Scatter(
    x=df['timestamp'],
    y=df['y'],
    mode='lines',
    line=dict(
        color='#b3b3b3',
        width=0.5
    )
)

fig = go.Figure(data=data, layout=layout)

fig.show()

## 3. Model

Define the forecasting horizon.

In [ ]:
forecast_period = 12 * 10 

Define a function for generating the forecasts and backcasts based on a set of hyperparameters.

In [ ]:
def nn(target, forecast_period, params):
    
    '''
    Parameters:
    _______________________________
    target: np.array, pd.Series, list.
        Time series.
        
    forecast_period: int.
        Forecasting horizon.
    
    params: dict.
        Model parameters.
    
    Returns:
    _______________________________
    results: pd.DataFrame
        Forecast and backcast.
    '''

    tf.random.set_seed(TENSORFLOW_SEED)

    model = NBeats(
        target,
        forecast_period=forecast_period,
        lookback_period=params['multiplier'] * forecast_period,
        stacks=['trend', 'seasonality'],
        num_trend_coefficients=params['num_trend_coefficients'],
        num_seasonal_coefficients=params['num_seasonal_coefficients'],
        hidden_units=params['hidden_units'],
        num_blocks_per_stack=params['num_blocks_per_stack'],
        share_weights=params['share_weights'],
        share_coefficients=params['share_coefficients'],
    )

    model.fit(
        learning_rate=params['learning_rate'],
        batch_size=params['batch_size'],
        epochs=params['epochs'],
        backcast_loss_weight=params['backcast_loss_weight'],
        loss='mae',
        validation_split=0,
        verbose=False
    )

    results = model.forecast(return_backcast=True)

    return results

Tune the hyperparameters.

In [ ]:
# Split the data into training and validation, set aside the last sequence for testing.
y_train = df['y'].iloc[: - 2 * forecast_period]
y_valid = df['y'].iloc[- 2 * forecast_period: - forecast_period] 

# Define the objective function.
def objective(trial):

    # Sample the hyperparameters.
    params = {
        'multiplier': trial.suggest_int('multiplier', 2, 7),
        'num_trend_coefficients': trial.suggest_int('num_trend_coefficients', 2, 4),
        'num_seasonal_coefficients': trial.suggest_int('num_seasonal_coefficients', 2, 8),
        'hidden_units': trial.suggest_int('hidden_units', 10, 100, step=10),
        'num_blocks_per_stack': trial.suggest_int('num_blocks_per_stack', 1, 3),
        'share_weights': trial.suggest_categorical('share_weights', [True, False]),
        'share_coefficients': trial.suggest_categorical('share_coefficients', [True, False]),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.01),
        'batch_size': trial.suggest_categorical('batch_size', [32, 64, 128, 256]),
        'epochs': trial.suggest_int('epochs', 100, 300, step=100),
        'backcast_loss_weight': trial.suggest_float('backcast_loss_weight', 0, 0.5, step=0.1),   
    }

    # Generate the forecast and backcast.
    df_pred = nn(
        target=y_train, 
        forecast_period=forecast_period, 
        params=params
    ) 

    # Extract the forecast.
    y_pred = df_pred['forecast'].iloc[- forecast_period:].values

    # Calculate the forecasting error.
    return np.mean(np.abs(y_valid - y_pred))

# Minimize the objective function.
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.RandomSampler(OPTUNA_SEED))
study.optimize(objective, n_trials=50)

# Extract the best parameters.
best_params = study.best_params

In [ ]:
best_params

Generate the test set predictions.

In [ ]:
df_test = nn(
    target=df['y'].iloc[: - forecast_period], 
    forecast_period=forecast_period, 
    params=best_params
) 

Add the timestamps to the output data frame.

In [ ]:
df_test['time_idx'] = df['timestamp']

In [ ]:
df_test.iloc[- 2 * forecast_period:]

Plot the test set predictions.

In [ ]:
layout = dict(
    width=800,
    height=425,
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(t=40, b=20, l=20, r=20),
    legend=dict(
        font=dict(
            color='#3a3a3a',
            size=10,
        ),
        orientation='h',
        x=0.0,
        y=1.1,
    ),
    xaxis=dict(
        type='date',
        nticks=20,
        tickfont=dict(
            color='#3a3a3a',
            size=10,
        ),
        linecolor='#d9d9d9',
        mirror=True,
        showgrid=False,
    ),
    yaxis=dict(
        tickfont=dict(
            color='#3a3a3a',
            size=10,
        ),
        linecolor='#d9d9d9',
        mirror=True,
        showgrid=False,
        zeroline=False,
    ),
)

data = []

data.append(
    go.Scatter(
        x=df['timestamp'],
        y=df['y'],
        name='Actual',
        mode='lines',
        line=dict(
            color='#b3b3b3',
            width=0.5
        )
    )
)

data.append(
    go.Scatter(
        x=df_test['time_idx'],
        y=df_test['forecast'],
        name='Forecast',
        mode='lines',
        line=dict(
            color='#0550ae',
            width=1
          )
      )
)

data.append(
    go.Scatter(
        x=df_test['time_idx'],
        y=df_test['backcast'],
        name='Backcast',
        mode='lines',
        line=dict(
            color='#8250df',
            width=1
        )
    )
)

fig = go.Figure(data=data, layout=layout)

fig.show()

## 4. Results

Fit the model using all the data.

In [ ]:
df_future = nn(
    target=df['y'], 
    forecast_period=forecast_period, 
    params=best_params
)

Add the timestamps to the output data frame.

In [ ]:
df_future['time_idx'].iloc[:- forecast_period] = df['timestamp']
df_future['time_idx'].iloc[- forecast_period:] = pd.date_range(start=df['timestamp'].iloc[-1], periods=1 + forecast_period, freq='M')[1:]

In [ ]:
df_future.iloc[- 2 * forecast_period:]

Plot the out of sample forecasts.

In [ ]:
layout = dict(
    width=800,
    height=425,
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(t=40, b=20, l=20, r=20),
    legend=dict(
        font=dict(
            color='#3a3a3a',
            size=10,
        ),
        orientation='h',
        x=0.0,
        y=1.1,
    ),
    xaxis=dict(
        type='date',
        nticks=20,
        tickfont=dict(
            color='#3a3a3a',
            size=10,
        ),
        linecolor='#d9d9d9',
        mirror=True,
        showgrid=False,
    ),
    yaxis=dict(
        tickfont=dict(
            color='#3a3a3a',
            size=10,
        ),
        linecolor='#d9d9d9',
        mirror=True,
        showgrid=False,
        zeroline=False,
    ),
)

data = []

data.append(
    go.Scatter(
        x=df['timestamp'],
        y=df['y'],
        name='Actual',
        mode='lines',
        line=dict(
            color='#b3b3b3',
            width=0.5
        )
    )
)

data.append(
    go.Scatter(
        x=df_future['time_idx'],
        y=df_future['forecast'],
        name='Forecast',
        mode='lines',
        line=dict(
            color='#0550ae',
            width=1
          )
      )
)

data.append(
    go.Scatter(
        x=df_future['time_idx'],
        y=df_future['backcast'],
        name='Backcast',
        mode='lines',
        line=dict(
            color='#8250df',
            width=1
        )
    )
)

fig = go.Figure(data=data, layout=layout)

fig.show()